In [280]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from rich import inspect
from tqdm.notebook import tqdm

import datetime
import numpy as np
import pandas as pd
import json

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)


## Load Data:


In [281]:
days_of_week = ["monday", "tuesday", "wednesday", "thursday", "friday"]

dfs = []

for day in days_of_week:
    df = pd.read_csv(
        f"../data/{day}.csv",
        parse_dates=["timestamp"],
        delimiter=";",
    )
    df["dow"] = day
    df["time"] = df.timestamp.dt.time
    df["customer_no"] = df["customer_no"].astype(str) + "_" + df["dow"]
    dfs.append(df)

df_all_days = pd.concat(dfs, ignore_index=True)

df_all_days = df_all_days.set_index("timestamp")

df_all_days.sample(5)


,customer_no,location,dow,time
timestamp,,,,
2019-09-04 07:41:00,49_wednesday,dairy,wednesday,07:41:00
2019-09-04 16:01:00,912_wednesday,dairy,wednesday,16:01:00
2019-09-05 13:06:00,601_thursday,dairy,thursday,13:06:00
2019-09-02 18:56:00,1156_monday,dairy,monday,18:56:00
2019-09-05 09:23:00,298_thursday,checkout,thursday,09:23:00


In [282]:
df_all_days.time.min(), df_all_days.time.max()


(datetime.time(7, 0), datetime.time(21, 50))

## Adding the period of day column, for analyzing data in an aggregate manner:

- morning: up to 12 pm
- afternoon: 12 to 4 pm
- evening: after 4 pm


In [283]:
def get_day_period(time):
    period = None
    if 7 <= time.hour < 12:
        period = "morning"
    elif 12 <= time.hour < 16:
        period = "afternoon"
    elif 16 <= time.hour:
        period = "evening"
    return period


In [284]:
df_all_days["day_period"] = df_all_days.time.apply(get_day_period)


## Resampling the Df to have a sample every minute:

Method used to fill unrecorded timestamps is "forward fill"


In [285]:
df_all_days = (
    df_all_days.groupby("customer_no")
    .resample("1Min")
    .ffill()
    .drop(columns="customer_no")
    .reset_index()
    .sort_values(by=["customer_no", "timestamp"])
)
df_all_days.sample(5)


,customer_no,timestamp,location,dow,time,day_period
43283,732_tuesday,2019-09-03 15:18:00,fruit,tuesday,15:18:00,afternoon
44031,751_tuesday,2019-09-03 15:55:00,drinks,tuesday,15:54:00,afternoon
40922,67_thursday,2019-09-05 07:36:00,checkout,thursday,07:36:00,morning
15270,1380_friday,2019-09-06 20:21:00,dairy,friday,20:19:00,evening
23776,255_friday,2019-09-06 09:13:00,fruit,friday,09:09:00,morning


## Adding a "checkout" entry of some customers whose end registered location was not checkout:


In [286]:
missing_checkout_entries = (
    []
)  # -- to be filled with the dicts of missing entries:
for entries, grp_data in df_all_days.groupby("customer_no"):
    if "entrance" not in grp_data.location.unique():
        # extract last row as a dict:
        checkout_entries = grp_data.iloc[0].to_dict()

        # the checkout timestamp is the last timestamp + 1 minute:
        entrance_timestamp = grp_data.iloc[0].timestamp + datetime.timedelta(
            minutes=-1
        )
        checkout_entries["timestamp"] = entrance_timestamp

        # update the location accordingly:
        checkout_entries["location"] = "entrance"
        missing_checkout_entries.append(checkout_entries)

    if "checkout" not in grp_data.location.unique():
        # extract last row as a dict:
        checkout_entries = grp_data.iloc[-1].to_dict()

        # the checkout timestamp is the last timestamp + 1 minute:
        checkout_timestamp = grp_data.iloc[-1].timestamp + datetime.timedelta(
            minutes=1
        )
        checkout_entries["timestamp"] = checkout_timestamp

        # update the location accordingly:
        checkout_entries["location"] = "checkout"
        missing_checkout_entries.append(checkout_entries)


In [287]:
# convert entries into df --> add it to the initial df --> re-sort the values per customer, per timestamp:
missing_entries_df = pd.DataFrame(missing_checkout_entries)

df_all_days = pd.concat([df_all_days, missing_entries_df], axis=0)

df_all_days = df_all_days.sort_values(by=["customer_no", "timestamp"])


## Adding a column for the previous timestamp's location:


In [288]:
df_all_days["from_location"] = (
    df_all_days["location"]
    .shift(1)
    .fillna("entrance")
    .replace("checkout", "entrance")
)

df_all_days.head(10)


,customer_no,timestamp,location,dow,time,day_period,from_location
0,1000_friday,2019-09-06 17:18:00,entrance,friday,17:19:00,evening,entrance
0,1000_friday,2019-09-06 17:19:00,fruit,friday,17:19:00,evening,entrance
1,1000_friday,2019-09-06 17:20:00,checkout,friday,17:20:00,evening,fruit
1,1000_monday,2019-09-02 17:43:00,entrance,monday,17:44:00,evening,entrance
2,1000_monday,2019-09-02 17:44:00,dairy,monday,17:44:00,evening,entrance
3,1000_monday,2019-09-02 17:45:00,dairy,monday,17:44:00,evening,dairy
4,1000_monday,2019-09-02 17:46:00,dairy,monday,17:44:00,evening,dairy
5,1000_monday,2019-09-02 17:47:00,dairy,monday,17:44:00,evening,dairy
6,1000_monday,2019-09-02 17:48:00,dairy,monday,17:44:00,evening,dairy
7,1000_monday,2019-09-02 17:49:00,dairy,monday,17:44:00,evening,dairy


In [289]:
path = "../data/cleaned_data.csv"
df_all_days.to_csv(path, index=False)


## EDA - General view of the number of customers per section


### Average customers per period per location:
- calculate the count of customers per hour for each day of the week.
- take the average count of customers per hour of day.


In [290]:
week_overview_per_section_per_period_per_dow = (
    df_all_days.groupby(["location", "dow", "day_period"])
    .timestamp.count()
    .reset_index()
    .rename(columns={"timestamp": "customers"})
)
week_overview_per_section_per_period = (
    week_overview_per_section_per_period_per_dow.groupby(
        ["location", "day_period"]
    )
    .customers.mean()
    .reset_index()
)

week_overview_per_section_per_period


,location,day_period,customers
0,checkout,afternoon,361.2
1,checkout,evening,656.8
2,checkout,morning,471.0
3,dairy,afternoon,902.8
4,dairy,evening,1470.4
5,dairy,morning,1181.6
6,drinks,afternoon,419.8
7,drinks,evening,928.8
8,drinks,morning,596.6
9,entrance,afternoon,362.2


In [291]:
week_overview_per_section_per_period.query(
    " day_period == 'afternoon' "
).customers.values


array([361.2, 902.8, 419.8, 362.2, 639.6, 313. ])

In [292]:
layout = dict(
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False),
    paper_bgcolor="rgba(0,0,0,0)",
    plot_bgcolor="rgba(0,0,0,0)",
    title_x=0.5,
    font=dict(color="white"),
)

title = "Average count of customers per location and period of day"
fig = go.Figure()

locations = week_overview_per_section_per_period.location.unique().tolist()
day_periods = ["morning", "afternoon", "evening"]

for period in day_periods:
    fig.add_trace(
        go.Bar(
            x=locations,
            y=week_overview_per_section_per_period.query(
                f" day_period == '{period}' "
            ).customers.values,
            name=period,
        )
    )

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(
    **layout, barmode="group", xaxis_tickangle=-45, title_text=title
)
fig.show()


### Entire Week overview (all data), per hour of the day:


In [293]:
df_all_days["hour_of_day"] = df_all_days.timestamp.dt.hour

week_overview_per_section_per_hour_per_dow = (
    df_all_days.groupby(["location", "dow", "hour_of_day"])
    .timestamp.count()
    .reset_index()
    .rename(columns={"timestamp": "customers"})
)
week_overview_per_section_per_hour = (
    week_overview_per_section_per_hour_per_dow.groupby(
        ["location", "hour_of_day"]
    )
    .customers.mean()
    .reset_index()
)

week_overview_per_section_per_hour.head(10)


,location,hour_of_day,customers
0,checkout,7,90.8
1,checkout,8,135.4
2,checkout,9,91.0
3,checkout,10,85.4
4,checkout,11,68.4
5,checkout,12,76.8
6,checkout,13,102.8
7,checkout,14,99.2
8,checkout,15,82.4
9,checkout,16,103.6


In [294]:
layout = dict(
    paper_bgcolor="rgba(0,0,0,0)",
    plot_bgcolor="rgba(0,0,0,0)",
    title_x=0.1,
    font=dict(color="white"),
)

fig = make_subplots(
    rows=len(locations),
    cols=1,
    subplot_titles=locations,
    shared_xaxes=True,
    shared_yaxes=True,
)

for i, location in enumerate(locations):
    df = week_overview_per_section_per_hour.query(f" location == '{location}' ")
    fig.add_trace(
        go.Bar(
            x=df.hour_of_day.values,
            y=df.customers.values,
            name=location,
            text=df.customers.values,
            textposition="auto",
        ),
        row=i + 1,
        col=1,
    )

title = "Average count of customers per location and hour of day"

y_max = week_overview_per_section_per_hour.customers.max()


fig.update_layout(
    **layout,
    title_text=title,
    height=1000,
    showlegend=False,
    # 1
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False, range=[0, y_max]),
    # 2
    xaxis2=dict(showgrid=False),
    yaxis2=dict(showgrid=False, range=[0, y_max]),
    # 3
    xaxis3=dict(showgrid=False),
    yaxis3=dict(showgrid=False, range=[0, y_max]),
    # 4
    xaxis4=dict(showgrid=False),
    yaxis4=dict(showgrid=False, range=[0, y_max]),
    # 5
    xaxis5=dict(showgrid=False),
    yaxis5=dict(showgrid=False, range=[0, y_max]),
    # 6
    xaxis6=dict(showgrid=False),
    yaxis6=dict(showgrid=False, range=[0, y_max]),
)

fig.show()


## Get the Transition Matrix from the data and the average # of customers at the entrance per minute:
- we need a transition matrix for each hour of the day
- we need a Poisson random generator to simulate the # of new customers that reach the entrance; each hour of the day will have its own Poisson
- each hour of the day has an hourly rate of customers at the entrance --> divide by 60 --> minutely rate of customers --> feed into Poisson

### Transition Matrix per hour of the day:


In [295]:
tm_hour = {}

entrance_hourly_customer_rate_per_hour = (
    week_overview_per_section_per_hour.query(" location == 'entrance' ")
)

for hour in df_all_days.hour_of_day.unique():
    if hour > 6:
        df_hour = df_all_days.query(f" hour_of_day == {hour} ")

        # get the Transition matrix per hour of day:
        tm = pd.crosstab(
            df_hour["from_location"],
            df_hour["location"],
            normalize="index",  # -- guarantees that each row's probas add up to 1
        ).to_dict(orient="index")

        lambda_hour = (
            entrance_hourly_customer_rate_per_hour.query(
                f" hour_of_day == {hour}"
            ).customers.values[0]
            / 60
        )
        # get the average # of customers per minute for each hour -- to be used as the lambda of the
        tm_hour[str(hour)] = {"tm": tm, "lambda_minute": lambda_hour}


## save the data as a json file:

In [296]:
with open('../data/tm_and_lambda_per_hour.json', 'w') as fp:
    json.dump(tm_hour, fp)